In [235]:
!pip install telegram-send

  Using cached telegram_send-0.25-py2.py3-none-any.whl (24 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Using cached colorama-0.4.4-py2.py3-none-any.whl (16 kB)
  Using cached python_telegram_bot-13.7-py3-none-any.whl (490 kB)
  Using cached tornado-6.1-cp37-cp37m-manylinux2010_x86_64.whl (428 kB)
  Using cached APScheduler-3.6.3-py2.py3-none-any.whl (58 kB)
  Using cached pytz-2021.1-py2.py3-none-any.whl (510 kB)
  Using cached certifi-2021.5.30-py2.py3-none-any.whl (145 kB)
  Using cached cachetools-4.2.2-py3-none-any.whl (11 kB)
  Using cached tzlocal-2.1-py2.py3-none-any.whl (16 kB)
  Using cached setuptools-57.4.0-py3-none-any.whl (819 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
ERROR: conda 4.8.2 requires ruamel_yaml>=0.11.14, which is not installed.
ERROR: tensorflow 2.1.0 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.5.0 which is incompatible.


In [231]:
def telegram_bot_sendtext(bot_message):
    
    bot_token = ''
    bot_chatID = ''
    send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message

    response = requests.get(send_text)

    return response.json()

In [232]:
import time

In [226]:
import ibmos2spark, os
# @hidden_cell

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_6337e1afa96b41479fda20829220b743 = 'https://s3.us.cloud-object-storage.appdomain.cloud'
else:
    endpoint_6337e1afa96b41479fda20829220b743 = 'https://s3.private.us.cloud-object-storage.appdomain.cloud'

credentials = {
    'endpoint': endpoint_6337e1afa96b41479fda20829220b743,
    'service_id': '',
    'iam_service_endpoint': 'https://iam.cloud.ibm.com/oidc/token',
    'api_key': '',
    'access_key': '',
    'secret_key': ''
}

configuration_name = 'spark_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

In [227]:
from pyspark.sql.types import *

schema = StructType() \
          .add("qid", IntegerType(), True) \
          .add("question", StringType(), True) \
          .add("vots", IntegerType(), True) \
          .add("answers", IntegerType(), True) \
          .add("views",  IntegerType(), True) \
          .add("tags", StringType(), True) \
          .add("created_at", TimestampType(), True) \
          .add("last_modifyed", TimestampType(), True)

In [ ]:
# apenas le os arquivos no bucket para dentro de um dataframe
bucket = ''
obj = '*.csv'
df = spark.read.options(delimiter=';',header=False).schema(schema).csv(cos.url(obj, bucket))

In [233]:
df.count()

In [ ]:
type(df)

In [ ]:
df.collect()[0]

In [ ]:
df.show()

In [ ]:
df.printSchema()

In [ ]:
from pyspark.sql.functions import udf
import re
def strArray_to_str(my_str):
    my_str = my_str[1:-1]
    my_str = re.sub(',',  '',    my_str)
    my_str = re.sub("'",  '', my_str)   
    #return my_str.split(" ")
    return ','.join(my_str.split(" "))

In [ ]:
str_to_arr_udf = udf(strArray_to_str,StringType())
df = df.withColumn("tags",str_to_arr_udf(df["tags"]))
df.show()

### save in a database

In [234]:
# @hidden_cell

# This connection object is used to access your data and contains your credentials or project token.
# You might want to remove those credentials before you share your notebook.

import os
from pyspark.sql import SparkSession
sparkSession = SparkSession(spark).builder.getOrCreate()

db2_connection_url = 'jdbc:db2://{}:{}/{}:sslConnection=true;'.format(
    f'{}.databases.appdomain.cloud',
    32733,
    'bludb'
)

In [ ]:
user = ''
password = """"""
prop = {"user":user, "password":password}

In [ ]:
df.write.jdbc(url=db2_connection_url , table='"BCZ79269"."SO_QUESTIONS"', mode="append",properties=prop)

### monitor 

In [ ]:
if questions.isStreaming == True:
    message = "🚀 Scraping\n\nO observador está de pé"
    telegram_bot_sendtext(message)    